<a href="https://colab.research.google.com/github/ogutiann/EnhancedPota/blob/main/SmallerEnhancedPOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch --upgrade --force-reinstall
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import os
import pickle
import math

# GPU diagnostics
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

# Configuration
DEMO_MODE = True
SAVE_DATA = True
if DEMO_MODE:
    N_WORKERS = 500
    N_TASKS = 1000
    RL_EPOCHS = 5
    VAE_EPOCHS = 10
else:
    N_WORKERS = 5000
    N_TASKS = 10000
    RL_EPOCHS = 20
    VAE_EPOCHS = 50

CAPACITY = 3
GPU_ACCELERATED = torch.cuda.is_available()
device = torch.device("cuda" if GPU_ACCELERATED else "cpu")

print(f"⚙️ Configuration: {N_WORKERS} workers, {N_TASKS} tasks")
print(f"🚀 Using {'GPU: ' + torch.cuda.get_device_name(0) if GPU_ACCELERATED else 'CPU'}")

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Load and preprocess air quality dataset
def load_air_quality_data():
    air_quality = None
    try:
        # Using the provided sample structure
        air_quality = pd.DataFrame({
            'geo_entity_name': ['Flushing and Whitestone (CD7)', 'Upper West Side (CD7)', 'Rockaway and Broad Channel (CD14)'],
            'data_valuemessage': [23.97, 27.42, 12.55]
        })

        # For full dataset, you would use:
        # air_quality = pd.read_csv('air_quality.csv')
        # air_quality = air_quality[['Geo Place Name', 'Data Value']]
        # air_quality.columns = ['geo_entity_name', 'data_valuemessage']

        print("✅ Loaded air quality data")
        print(f"Found {len(air_quality)} unique geographic entities")
        return air_quality
    except Exception as e:
        print(f"⚠️ Air quality data loading failed: {e}")
        return None

AIR_QUALITY_DF = load_air_quality_data()

# Data persistence
def save_synthetic_data(workers, tasks, filename="synthetic_data.pkl"):
    with open(filename, 'wb') as f:
        pickle.dump({'workers': workers, 'tasks': tasks}, f)

def load_synthetic_data(filename="synthetic_data.pkl"):
    try:
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            return data['workers'], data['tasks']
    except FileNotFoundError:
        return None

######################
### 1. System Model ##
######################
class Worker:
    def __init__(self, id, real_world=False):
        self.id = id
        self.real_world = real_world

        if real_world and AIR_QUALITY_DF is not None:
            self.location = self.generate_nyc_location()
            self.skill = self.generate_nyc_skills()
            self.speed = self.generate_nyc_speed()
        else:
            self.location = np.array([np.random.uniform(0,100), np.random.uniform(0,100)])
            self.skill = np.array([np.random.uniform(2,5), np.random.uniform(2,5)])
            self.speed = np.random.uniform(0.5, 2)

        self.weights = np.random.dirichlet(np.ones(3))
        self.reputation = np.random.uniform(0.7, 0.95)
        self.utility_history = []
        self.assigned_tasks = []

    def generate_nyc_location(self):
        borough_probs = [0.3, 0.3, 0.2, 0.15, 0.05]
        borough = np.random.choice(5, p=borough_probs)
        centroids = {
            0: [40.7831, -73.9712],
            1: [40.6782, -73.9442],
            2: [40.7282, -73.7949],
            3: [40.8448, -73.8648],
            4: [40.5795, -74.1502]
        }
        lat = centroids[borough][0] + np.random.uniform(-0.05, 0.05)
        lon = centroids[borough][1] + np.random.uniform(-0.05, 0.05)
        return np.array([lat, lon])

    def generate_nyc_skills(self):
        if self.location[0] > 40.75:
            return np.array([np.random.uniform(4, 5), np.random.uniform(4, 5)])
        elif self.location[0] > 40.65:
            return np.array([np.random.uniform(3.5, 4.5), np.random.uniform(3.5, 4.5)])
        else:
            return np.array([np.random.uniform(3, 4), np.random.uniform(3, 4)])

    def generate_nyc_speed(self):
        if self.location[0] > 40.75:
            return np.random.uniform(0.5, 1.2)
        elif self.location[0] > 40.65:
            return np.random.uniform(1.0, 1.8)
        else:
            return np.random.uniform(1.5, 2.0)

    def travel_time(self, task_location):
        # Haversine distance calculation
        lat1, lon1 = np.radians(self.location)
        lat2, lon2 = np.radians(task_location)

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance = 6371 * c  # Earth radius in km

        return max(0.1, distance / self.speed)

class Task:
    def __init__(self, id, real_world=False):
        self.id = id
        self.real_world = real_world

        if real_world and AIR_QUALITY_DF is not None:
            self.location, self.reward, self.difficulty, self.required_skill = self.generate_air_quality_task()
        else:
            self.location = np.array([np.random.uniform(0,100), np.random.uniform(0,100)])
            self.reward = np.random.uniform(5,20)
            self.difficulty = np.random.uniform(1,10)
            self.required_skill = np.array([np.random.uniform(1,4), np.random.uniform(1,4)])

        self.deadline = np.random.randint(10,60)

    def generate_air_quality_task(self):
        """Generate task based on NYC air quality data"""
        borough_centroids = {
            "Manhattan": [40.7831, -73.9712],
            "Brooklyn": [40.6782, -73.9442],
            "Queens": [40.7282, -73.7949],
            "Bronx": [40.8448, -73.8648],
            "Staten Island": [40.5795, -74.1502]
        }

        # Select random geographic entity
        geo_entity = np.random.choice(AIR_QUALITY_DF['geo_entity_name'])
        b_data = AIR_QUALITY_DF[AIR_QUALITY_DF['geo_entity_name'] == geo_entity].iloc[0]
        air_quality_value = b_data['data_valuemessage']

        # Normalize air quality values to [1,10] range
        aq_min, aq_max = AIR_QUALITY_DF['data_valuemessage'].min(), AIR_QUALITY_DF['data_valuemessage'].max()
        normalized_aq = (air_quality_value - aq_min) / (aq_max - aq_min) * 9 + 1
        difficulty = np.clip(normalized_aq, 1, 10)

        # Map to nearest borough
        min_dist = float('inf')
        closest_borough = "Manhattan"
        for borough, centroid in borough_centroids.items():
            dist = np.linalg.norm(np.array(centroid) - np.array([40.7128, -74.0060]))
            if dist < min_dist:
                min_dist = dist
                closest_borough = borough

        location = np.array(borough_centroids[closest_borough])

        # Cap required skills to worker capabilities
        required_skill = np.array([3.0 + difficulty/3, 3.5 + difficulty/3])
        required_skill = np.clip(required_skill, 0, 5)  # Cap at max worker skill

        reward = 10 + difficulty * 2
        return location, reward, difficulty, required_skill

################################
### 2. Deep-Quality Reputation ##
################################
class QualityVAE(nn.Module):
    def __init__(self, input_dim=3, latent_dim=8):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32), nn.ReLU(),
            nn.Linear(32, 16), nn.ReLU()
        )
        self.mu = nn.Linear(16, latent_dim)
        self.logvar = nn.Linear(16, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16), nn.ReLU(),
            nn.Linear(16, 32), nn.ReLU(),
            nn.Linear(32, input_dim), nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.mu(h), self.logvar(h)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + 0.5 * KLD

################################################
### 3. Federated RL for Preference Optimization #
################################################
class LSTMPolicy(nn.Module):
    def __init__(self, input_size=5, hidden_size=32, output_size=3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])
        return torch.softmax(x, dim=-1)

def federated_average(models):
    global_dict = {}
    for key in models[0].state_dict().keys():
        global_dict[key] = torch.stack([m.state_dict()[key] for m in models]).mean(0)
    return global_dict

def train_worker_rl(worker, tasks, epochs=RL_EPOCHS, batch_size=64, lr=0.005):
    model = LSTMPolicy().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    n_samples = 1000 if not DEMO_MODE else 500
    states = torch.randn(n_samples, 1, 5, device=device)
    actions = torch.randint(0, 3, (n_samples,), device=device)
    rewards = torch.rand(n_samples, device=device)

    for epoch in range(epochs):
        perm = torch.randperm(n_samples)
        for i in range(0, n_samples, batch_size):
            idx = perm[i:i+batch_size]
            batch_s = states[idx]
            batch_a = actions[idx]
            batch_r = rewards[idx]

            action_probs = model(batch_s)
            selected_probs = action_probs[torch.arange(len(batch_a)), batch_a]
            loss = -torch.log(selected_probs) * batch_r
            loss = loss.mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model

#############################################
### 4. Vectorized Utility Calculations (Fixed) ##
#############################################
def vectorized_utilities(workers, tasks, beta=0.3, zeta=0.3, kappa=0.7):
    worker_locs = np.array([w.location for w in workers])
    task_locs = np.array([t.location for t in tasks])
    n_workers = len(workers)
    n_tasks = len(tasks)

    # Initialize distance matrix
    distances = np.zeros((n_workers, n_tasks))

    # Calculate distances
    for i, worker in enumerate(workers):
        for j, task in enumerate(tasks):
            if worker.real_world:
                # Haversine distance for real-world locations
                lat1, lon1 = np.radians(worker.location)
                lat2, lon2 = np.radians(task.location)
                dlat = lat2 - lat1
                dlon = lon2 - lon1
                a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
                c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
                distances[i, j] = 6371 * c
            else:
                # Euclidean distance for synthetic data
                distances[i, j] = np.linalg.norm(worker.location - task.location)

    # Vectorized calculations
    speeds = np.array([w.speed for w in workers])
    tau = distances / speeds[:, None]  # Shape: (n_workers, n_tasks)

    worker_skills = np.array([w.skill for w in workers])
    task_reqs = np.array([t.required_skill for t in tasks])

    # Skill gap calculation
    skill_gap = np.zeros((n_workers, n_tasks))
    for i in range(n_workers):
        for j in range(n_tasks):
            skill_gap[i, j] = np.mean(worker_skills[i] - task_reqs[j])

    worker_weights = np.array([w.weights for w in workers])
    task_rewards = np.array([t.reward for t in tasks])
    reputations = np.array([w.reputation for w in workers])

    # Worker utilities
    w_utils = (
        worker_weights[:, 0][:, None] * (1 / (tau + 1e-8)) +
        worker_weights[:, 1][:, None] * task_rewards[None, :] +
        worker_weights[:, 2][:, None] * skill_gap +
        beta * reputations[:, None]
    )

    # Task utilities
    skill_mask = np.zeros((n_workers, n_tasks), dtype=bool)
    for i in range(n_workers):
        for j in range(n_tasks):
            skill_mask[i, j] = np.all(worker_skills[i] >= task_reqs[j])

    t_utils = kappa * (1 / (tau + 1e-8)) + zeta * reputations[:, None]
    t_utils = np.where(skill_mask, t_utils, -np.inf)

    return w_utils, t_utils

#############################################
### 5. Matching Algorithms Implementations ##
#############################################
def rags_matching(workers, tasks, capacity=CAPACITY, beta=0.3, zeta=0.3, kappa=0.7):
    w_utils, t_utils = vectorized_utilities(workers, tasks, beta, zeta, kappa)
    n_workers = len(workers)
    n_tasks = len(tasks)

    # Initialize data structures
    proposals = np.zeros(n_workers, dtype=int)
    worker_assignments = [[] for _ in range(n_workers)]
    task_assignments = [[] for _ in range(n_tasks)]
    unmatched = list(range(n_workers))

    # Precompute preferences
    w_prefs = np.argsort(-w_utils, axis=1)  # Worker preferences: tasks sorted by utility

    # Matching loop
    while unmatched:
        i = unmatched.pop(0)
        if proposals[i] >= n_tasks:
            continue

        j = w_prefs[i, proposals[i]]
        proposals[i] += 1

        if len(task_assignments[j]) < capacity:
            worker_assignments[i].append(j)
            task_assignments[j].append(i)
        else:
            current_workers = task_assignments[j]
            current_utils = [t_utils[k, j] for k in current_workers]
            min_util = min(current_utils)
            worst_idx = current_workers[np.argmin(current_utils)]

            if t_utils[i, j] > min_util:
                # Replace worker
                worker_assignments[worst_idx].remove(j)
                task_assignments[j].remove(worst_idx)
                unmatched.append(worst_idx)

                worker_assignments[i].append(j)
                task_assignments[j].append(i)
            else:
                unmatched.append(i)

    return worker_assignments, task_assignments

def classic_gs(workers, tasks, capacity=CAPACITY):
    """True Gale-Shapley implementation without reputation"""
    n_workers = len(workers)
    n_tasks = len(tasks)

    # Precompute utilities
    w_utils = np.zeros((n_workers, n_tasks))
    t_utils = np.zeros((n_tasks, n_workers))

    for i, worker in enumerate(workers):
        for j, task in enumerate(tasks):
            # Worker utility without reputation
            tau = worker.travel_time(task.location)
            skill_gap = np.mean(worker.skill - task.required_skill)
            w_utils[i, j] = (worker.weights[0] * (1/tau) +
                             worker.weights[1] * task.reward +
                             worker.weights[2] * skill_gap)

            # Task utility without reputation
            if np.all(worker.skill >= task.required_skill):
                t_utils[j, i] = 1/tau
            else:
                t_utils[j, i] = -np.inf

    # Preference lists
    w_prefs = [np.argsort(-w_utils[i]) for i in range(n_workers)]
    t_prefs = [np.argsort(-t_utils[j]) for j in range(n_tasks)]

    # Initialize
    proposals = np.zeros(n_workers, dtype=int)
    worker_assignments = [[] for _ in range(n_workers)]
    task_assignments = [[] for _ in range(n_tasks)]
    unmatched = list(range(n_workers))

    # Matching loop
    while unmatched:
        i = unmatched.pop(0)
        if proposals[i] >= n_tasks:
            continue

        j = w_prefs[i][proposals[i]]
        proposals[i] += 1

        if len(task_assignments[j]) < capacity:
            worker_assignments[i].append(j)
            task_assignments[j].append(i)
        else:
            # Find worst assigned worker
            current_utils = [t_utils[j, k] for k in task_assignments[j]]
            min_util = min(current_utils)
            worst_idx = task_assignments[j][np.argmin(current_utils)]

            if t_utils[j, i] > min_util:
                # Replace worker
                worker_assignments[worst_idx].remove(j)
                task_assignments[j].remove(worst_idx)
                unmatched.append(worst_idx)

                worker_assignments[i].append(j)
                task_assignments[j].append(i)
            else:
                unmatched.append(i)

    return worker_assignments, task_assignments

def random_matching(workers, tasks, capacity=CAPACITY):
    worker_assignments = [[] for _ in range(len(workers))]
    task_assignments = [[] for _ in range(len(tasks))]
    all_pairs = [(i, j) for i in range(len(workers)) for j in range(len(tasks))]
    random.shuffle(all_pairs)

    for i, j in all_pairs:
        if (len(worker_assignments[i]) < capacity and
            len(task_assignments[j]) < capacity and
            np.all(workers[i].skill >= tasks[j].required_skill)):
            worker_assignments[i].append(j)
            task_assignments[j].append(i)

    return worker_assignments, task_assignments

def fml_matching(workers, tasks, capacity=CAPACITY):
    avg_weights = np.mean([w.weights for w in workers], axis=0)
    for worker in workers:
        worker.weights = avg_weights.copy()
    return classic_gs(workers, tasks, capacity)

############################
### 6. Evaluation Metrics ##
############################
def calculate_satisfaction(workers, tasks, assignments):
    total_util = 0
    count = 0
    for i, worker in enumerate(workers):
        if assignments[i]:
            utils = [worker_utility(worker, tasks[j]) for j in assignments[i]]
            total_util += np.mean(utils)
            count += 1
    return total_util / count if count else 0

def worker_utility(worker, task):
    tau = worker.travel_time(task.location)
    skill_gap = np.mean(worker.skill - task.required_skill)
    return (worker.weights[0] * (1/tau) +
            worker.weights[1] * task.reward +
            worker.weights[2] * skill_gap +
            0.3 * worker.reputation)

def calculate_task_coverage(task_assignments):
    return sum(len(a) > 0 for a in task_assignments) / len(task_assignments)

def calculate_quality(workers, tasks, assignments):
    qualities = []
    for i, worker in enumerate(workers):
        for task_id in assignments[i]:
            task = tasks[task_id]
            skill_diff = worker.skill - task.required_skill
            # Only consider positive skill differences
            base_quality = np.mean(np.maximum(skill_diff, 0)) / 5
            noise = np.random.normal(0, 0.2 * (1 - worker.reputation))
            qualities.append(max(0, min(1, base_quality + noise)))
    return np.mean(qualities) if qualities else 0

def check_stability(workers, tasks, worker_assignments, task_assignments):
    blocking_pairs = 0
    sample_size = min(500, len(workers))

    for i in np.random.choice(len(workers), sample_size, replace=False):
        worker = workers[i]
        current_utils = [worker_utility(worker, tasks[j]) for j in worker_assignments[i]]
        current_min = min(current_utils) if current_utils else -np.inf

        for j in np.random.choice(len(tasks), min(50, len(tasks)), replace=False):
            if j in worker_assignments[i]:
                continue
            w_util = worker_utility(worker, tasks[j])
            if w_util <= current_min:
                continue
            t_util = task_utility(tasks[j], worker)
            current_workers = task_assignments[j]
            if not current_workers:
                blocking_pairs += 1
                continue
            min_current_util = min(task_utility(tasks[j], workers[k]) for k in current_workers)
            if t_util > min_current_util:
                blocking_pairs += 1

    return 1 - (blocking_pairs / (sample_size * 50))

def task_utility(task, worker):
    if np.any(worker.skill < task.required_skill):
        return 0
    tau = worker.travel_time(task.location)
    return 0.7 * (1/tau) + 0.3 * worker.reputation

########################
### 7. Main Simulation #
########################
def run_full_simulation(real_world=False):
    print(f"🚀 Starting {'real-world' if real_world else 'synthetic'} simulation")
    results = []
    methods = {
        "POTA (Proposed)": rags_matching,
        "Classic GS": classic_gs,
        "Random Allocation": random_matching,
        "FML": fml_matching
    }

    # Generate or load data
    if not real_world and SAVE_DATA:
        data = load_synthetic_data()
        if data:
            workers, tasks = data
            print("✅ Loaded saved synthetic data")
        else:
            workers = [Worker(i, real_world) for i in tqdm(range(N_WORKERS), desc="Workers")]
            tasks = [Task(j, real_world) for j in tqdm(range(N_TASKS), desc="Tasks")]
            save_synthetic_data(workers, tasks)
    else:
        workers = [Worker(i, real_world) for i in tqdm(range(N_WORKERS), desc="Workers")]
        tasks = [Task(j, real_world) for j in tqdm(range(N_TASKS), desc="Tasks")]

    # Train DQRS model
    print("Training DQRS VAE...")
    vae_model = QualityVAE().to(device)
    optimizer = optim.Adam(vae_model.parameters(), lr=0.001)
    n_samples = 10000 if DEMO_MODE else 100000
    features = torch.randn(n_samples, 3, device=device)

    for _ in tqdm(range(VAE_EPOCHS), desc="VAE Training"):
        recon, mu, logvar = vae_model(features)
        loss = vae_loss(recon, features, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Federated RL Training
    print("Training federated RL agents...")
    worker_models = []
    batch_size = min(100, len(workers))

    # FIXED: Removed extra parenthesis in tqdm call
    for i in tqdm(range(0, len(workers), batch_size), desc="Federated RL"):
        batch = workers[i:i+batch_size]
        batch_models = [train_worker_rl(w, tasks) for w in batch]
        worker_models.extend(batch_models)
        if GPU_ACCELERATED:
            torch.cuda.empty_cache()

    # Apply federated averaging
    print("Applying federated averaging...")
    global_weights = federated_average(worker_models)
    for i, model in enumerate(worker_models):
        model.load_state_dict(global_weights)
        with torch.no_grad():
            input_tensor = torch.randn(1, 1, 5)
            if GPU_ACCELERATED:
                input_tensor = input_tensor.to(device)
            weights = model(input_tensor).cpu().numpy()[0]
            workers[i].weights = weights

    # Run all matching algorithms
    for method_name, matching_fn in methods.items():
        print(f"\n🔍 Running {method_name}...")
        start_time = time.time()
        for w in workers:
            w.assigned_tasks = []
        worker_assignments, task_assignments = matching_fn(workers, tasks)
        elapsed = time.time() - start_time

        # Calculate metrics
        satisfaction = calculate_satisfaction(workers, tasks, worker_assignments)
        coverage = calculate_task_coverage(task_assignments)
        quality = calculate_quality(workers, tasks, worker_assignments)
        stability = check_stability(workers, tasks, worker_assignments, task_assignments)

        results.append({
            "Method": method_name,
            "Satisfaction": satisfaction,
            "Task Coverage": coverage,
            "Data Quality": quality,
            "Stability": stability,
            "Time (s)": elapsed,
            "Scenario": "Real-world" if real_world else "Synthetic"
        })

        print(f"  Satisfaction: {satisfaction:.3f} | Coverage: {coverage:.1%} | "
              f"Quality: {quality:.3f} | Stability: {stability:.1%} | Time: {elapsed:.1f}s")

    return results

def run_parameter_sensitivity():
    print("🔬 Running parameter sensitivity analysis...")
    results = []
    workers = [Worker(i) for i in range(1000)]
    tasks = [Task(j) for j in range(2000)]

    # Test beta (reputation weight)
    for beta in [0.1, 0.2, 0.3, 0.4, 0.5]:
        start_time = time.time()
        worker_assignments, _ = rags_matching(workers, tasks, beta=beta)
        elapsed = time.time() - start_time
        satisfaction = calculate_satisfaction(workers, tasks, worker_assignments)
        quality = calculate_quality(workers, tasks, worker_assignments)
        results.append({"Parameter": "β", "Value": beta, "Satisfaction": satisfaction, "Quality": quality, "Time (s)": elapsed})

    # Test learning rate (eta)
    for lr in [0.001, 0.005, 0.01, 0.05, 0.1]:
        worker_models = [train_worker_rl(w, tasks, lr=lr) for w in workers]
        global_weights = federated_average(worker_models)
        for i, model in enumerate(worker_models):
            model.load_state_dict(global_weights)
            with torch.no_grad():
                input_tensor = torch.randn(1, 1, 5)
                if GPU_ACCELERATED:
                    input_tensor = input_tensor.to(device)
                weights = model(input_tensor).cpu().numpy()[0]
                workers[i].weights = weights
        start_time = time.time()
        worker_assignments, _ = rags_matching(workers, tasks)
        elapsed = time.time() - start_time
        satisfaction = calculate_satisfaction(workers, tasks, worker_assignments)
        quality = calculate_quality(workers, tasks, worker_assignments)
        results.append({"Parameter": "η", "Value": lr, "Satisfaction": satisfaction, "Quality": quality, "Time (s)": elapsed})

    # Test alpha (reputation decay)
    original_reps = [w.reputation for w in workers]
    for alpha in [0.6, 0.7, 0.8, 0.9, 0.95]:
        for w in workers:
            w.reputation *= alpha
        start_time = time.time()
        worker_assignments, _ = rags_matching(workers, tasks)
        elapsed = time.time() - start_time
        satisfaction = calculate_satisfaction(workers, tasks, worker_assignments)
        quality = calculate_quality(workers, tasks, worker_assignments)
        results.append({"Parameter": "α", "Value": alpha, "Satisfaction": satisfaction, "Quality": quality, "Time (s)": elapsed})
        for w, rep in zip(workers, original_reps):
            w.reputation = rep

    return pd.DataFrame(results)

def plot_results(results_df, sensitivity_df, scenario):
    plt.figure(figsize=(14, 10))
    plt.subplot(2, 2, 1)
    sns.barplot(x="Method", y="Satisfaction", data=results_df)
    plt.title(f"Worker Satisfaction ({scenario})")
    plt.ylim(0, 10)
    plt.xticks(rotation=15)

    plt.subplot(2, 2, 2)
    sns.barplot(x="Method", y="Data Quality", data=results_df)
    plt.title(f"Data Quality ({scenario})")
    plt.ylim(0, 1)
    plt.xticks(rotation=15)

    plt.subplot(2, 2, 3)
    sns.barplot(x="Method", y="Task Coverage", data=results_df)
    plt.title(f"Task Coverage ({scenario})")
    plt.ylim(0, 1)
    plt.xticks(rotation=15)

    plt.subplot(2, 2, 4)
    sns.barplot(x="Method", y="Stability", data=results_df)
    plt.title(f"Matching Stability ({scenario})")
    plt.ylim(0, 1)
    plt.xticks(rotation=15)

    plt.tight_layout()
    plt.savefig(f"comparison_{scenario.lower().replace(' ', '_')}.png")

    plt.figure(figsize=(12, 8))
    for i, param in enumerate(['β', 'η', 'α']):
        plt.subplot(2, 2, i+1)
        param_df = sensitivity_df[sensitivity_df['Parameter'] == param]
        plt.plot(param_df['Value'], param_df['Satisfaction'], 'o-', label='Satisfaction')
        plt.plot(param_df['Value'], param_df['Quality'], 's-', label='Quality')
        plt.title(f"Sensitivity to {param}")
        plt.xlabel("Parameter Value")
        plt.ylabel("Metric Value")
        plt.legend()
        plt.grid(True)
    plt.tight_layout()
    plt.savefig("parameter_sensitivity.png")
    print("\n📊 Visualizations saved")

# Main execution
if __name__ == "__main__":
    start_time = time.time()

    # Run synthetic scenario
    synthetic_results = run_full_simulation(real_world=False)
    synthetic_df = pd.DataFrame(synthetic_results)

    # Run real-world scenario
    realworld_results = run_full_simulation(real_world=True)
    realworld_df = pd.DataFrame(realworld_results)

    # Run sensitivity analysis
    sensitivity_df = run_parameter_sensitivity()

    # Combine and save results
    results_df = pd.concat([synthetic_df, realworld_df])
    results_df.to_csv("pota_results.csv", index=False)
    sensitivity_df.to_csv("parameter_sensitivity.csv", index=False)
    print("\n✅ Results saved to CSV files")

    # Visualize results
    plot_results(synthetic_df, sensitivity_df, "Synthetic")
    plot_results(realworld_df, sensitivity_df, "Real-world")

    # Display final results
    print("\n⭐ Final Results ⭐")
    print(results_df.groupby(['Scenario', 'Method']).mean())
    print(f"\n🕒 Total execution time: {time.time()-start_time:.1f} seconds")


  Using cached torch-2.8.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27